# Model development and training
In this notebook we will develop a Recurrent Neural Network (RNN) using LSTM nodes, so as to predict the next character, given a sequence of 100 characters.

First of all, we import the numpy and keras modules, important for storing data and defining the model respectively.

In [1]:
#import necessary libraries
import numpy as np
from keras.utils import np_utils
import sys

# Loading the data

Next, we download the training data. The children book "Alice's Adventures in Wonderland" written by Lewis Caroll has been used as the training data in this project. The ebook can be downloaded from [here](http://www.gutenberg.org/ebooks/11?msg=welcome_stranger) in Plain Text UTF-8 format. The downloaded book has been stored in the root directory with the name 'wonderland.txt'. We open this book using the open command and convert all characters into lowercase (so as to reduce the number of characters in the vocabulary, making it easier to learn for the model.)

In [2]:
file = open('wonderland.txt', encoding = 'utf8')
raw_text = file.read()    #you need to read further characters as well
raw_text = raw_text.lower()

In [3]:
raw_text

'\ufeffproject gutenberg’s alice’s adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.org\n\n\ntitle: alice’s adventures in wonderland\n\nauthor: lewis carroll\n\nposting date: june 25, 2008 [ebook #11]\nrelease date: march, 1994\nlast updated: october 6, 2016\n\nlanguage: english\n\ncharacter set encoding: utf-8\n\n*** start of this project gutenberg ebook alice’s adventures in wonderland ***\n\n\n\n\n\n\n\n\n\n\nalice’s adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 3.0\n\n\n\n\nchapter i. down the rabbit-hole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into the\nbook her sister was reading, but it had no pictures or convers

Next, we store all the distinct characters occurying in the book in the chars variable. We also remove some of the rare characters (stored in bad-chars) from the book. The final vocabulary of the book is printed at the end of code segment.

# Text Pre-Processing

In [4]:
# Storing all the unique characters present in the text
chars = sorted(list(set(raw_text)))
print(chars)

bad_chars = ['#', '*', '@', '_', '\ufeff']
for i in range(len(bad_chars)):
    raw_text = raw_text.replace(bad_chars[i],"")

chars = sorted(list(set(raw_text)))
print(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']
['\n', ' ', '!', '$', '%', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”']


Next, we summarize the entire book to find that the book consists of a total of 163,721 characters (which is relatively small) and the final number of characters in the vocabulary is 56.

In [5]:
text_length = len(raw_text)
char_length = len(chars)
VOCABULARY = char_length
print("Text length = " + str(text_length))
print("No. of characters = " + str(char_length))

Text length = 163721
No. of characters = 56


### creating sliding window  

Now, we need to create our dataset in the form the model will need. So, we create an input window of 100 characters (SEQ_LENGTH = 100) and shift the window one character at a time until we reach the end of the book. An encoding is used, so as to map each of the characters into it's corresponding location in the vocabulary. Each time the input window contains a new sequence, it is converted into integers, using this encoding and appended to the input list data_X. For all such input windows, the character just following the sequence is appended to the output list data_Y.

In [6]:
SEQ_LENGTH = 100
# Creating dictionaries to map each character to an index 
char_to_int = dict((c, i) for i, c in enumerate(chars))
data_X = []
data_Y = []

for i in range(len(raw_text) - SEQ_LENGTH):
    X_text = raw_text[i: i + SEQ_LENGTH]
    X = [char_to_int[char] for char in X_text]
    data_X.append(X)    
    Y = raw_text[i + SEQ_LENGTH]
    data_Y.append(char_to_int[Y])

Now, the data_X and data_Y lists are converted into a numpy array of required dimensions, so that they can be fed to the model for the training.

In [7]:
#reshape X to be [samples, time steps, features]
length = len(data_X)
data_X = np.array(data_X)
data_X = np.reshape(data_X, (data_X.shape[0], data_X.shape[1], 1))
data_X = data_X/float(VOCABULARY)

data_Y = np.array(data_Y)
# one hot encode the output variable
data_Y = np_utils.to_categorical(data_Y)
print(data_X.shape)
print(data_Y.shape)

(163621, 100, 1)
(163621, 56)


## Model Building

In [8]:
#importing necessary libraries
import keras
from keras.models import Sequential #Initialise the model
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint


###### We define the model now. The model is given an input of 100 character sequences and it outputs the respective probabilities with which a character can succeed the input sequence. The model consists of 3 hidden layers. The first two hidden layers consist of 256 LSTM cells, and the second layer is fully connected to the third layer. The number of neurons in the third layer is same as the number of unique characters in the training set. The neurons in the third layer, use softmax activation so as to convert their outputs into respective probabilities. The loss used is Categorical cross entropy and the optimizer used is Adam.

In [9]:
#Define the LSTM Model
model = Sequential()
model.add(LSTM(256, input_shape = (SEQ_LENGTH, 1), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(VOCABULARY, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

Now, finally the model is build and then fitted for training. The model is trained for 50 epochs and a batch size of 128 sequences has been used. After every epoch, the current model state is saved if the model has the least loss encountered till that time. We can see that the training loss decreases in almost every epoch till 50 epochs and probably there is further scope to improve the model.

The total training time is ~90 hours on a CPU and ~12 hours if performed on a GPU.

In [ ]:
#Define the check point
filepath="text_generation.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

history = model.fit(data_X, data_Y, epochs =10 , batch_size = 128, callbacks = callbacks_list)

Epoch 1/10
1279/1279 [==============================] - ETA: 0s - loss: 3.0693 
Epoch 00001: loss improved from inf to 3.06929, saving model to text_generation.h5
1279/1279 [==============================] - 21201s 17s/step - loss: 3.0693
Epoch 2/10
1279/1279 [==============================] - ETA: 0s - loss: 2.8906
Epoch 00002: loss improved from 3.06929 to 2.89058, saving model to text_generation.h5
1279/1279 [==============================] - 4777s 4s/step - loss: 2.8906
Epoch 3/10
1279/1279 [==============================] - ETA: 0s - loss: 2.6854
Epoch 00003: loss improved from 2.89058 to 2.68539, saving model to text_generation.h5
1279/1279 [==============================] - 4269s 3s/step - loss: 2.6854
Epoch 4/10
1279/1279 [==============================] - ETA: 0s - loss: 2.5546
Epoch 00004: loss improved from 2.68539 to 2.55455, saving model to text_generation.h5
1279/1279 [==============================] - 2756s 2s/step - loss: 2.5546
Epoch 5/10
1279/1279 [===================

Now that our model has been trained, we can use it for different generating texts as well as predicting next word, .The best model with least loss as we obtained in the last epoch of training is loaded and the model is build and recompiled.

## Generating text

In [85]:
#load the network weights
filename = 'text_generation.h5'
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

The initital 100 character seed used for generating text are the first few characters of the famous children book 'The Cat in the Hat' by Dr. Seuss 

In [86]:
initial_text = ' the sun did not shine, it was too wet to play, so we sat in the house all that cold, cold wet day. 
'# we sat here we two and we said how we wish we had something to do.'
initial_text = [char_to_int[c] for c in initial_text]

Starting with the initial seed next 100 characters are generated by shifting the 100 character input window for generating the next character.

In [87]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
GENERATED_LENGTH = 100
test_text = initial_text
#generated_text = []
# generate characters
for i in range(100):
    X = np.reshape(test_text, (1, SEQ_LENGTH, 1))
    X  = X  / float(VOCABULARY)
    Prediction = model.predict(X)
    index = np.argmax(Prediction)
    result = int_to_char[index]
    #generated_text.append(int_to_char[index])
    sys.stdout.write(result)
    test_text.append(index)
    test_text = test_text[1:]
print ("\nDone.")

and the dout and the wer  and the wer  and the dout and the wer  and the wer  and the dout and the w
Done.


In [29]:
print(''.join(generated_text))

and the dout and the wer  and the wer  and the dout and the wer  and the wer  and the dout and the w


In [24]:
len(generated_text)

1000